# IE 306.02 SimPy Assingment 1
## Group 24
- Özge Dinçsoy
- Alperen Yakut
- Mustafa Enes Çakır

**NOTE: We write required statistics programatically, so you need to do "Cell->Run All" (or run all cells from top to bottom manuelly) in order to see correct results.**

**Requirements:** Python 3.6

In [1]:
import simpy
import random

**Define a set of globals that define the characteristics of the model instance to be simulated.**
- *RANDOM_SEED* **->** The seed for the random number generators
- *INTERARRIVAL_MEAN* **->** The mean of exponentially distributed arrival time
- *INTERARRIVAL_RATE* **->** The rate of exponentially distributed arrival time
- *SERVICE_TIME_OPERATOR_MEAN* **->** The mean of Gaussian distributed operator service time
- *SERVICE_TIME_OPERATOR_DEVIATION* **->** The standard deviation of Gaussian distributed operator service time
- *SERVICE_TIME_EXPERT_MEAN* **->** The mean of Exponentially distributed expert service time
- *SERVICE_TIME_EXPERT_DEVIATION* **->** The standard deviation of Normally distributed expert service time
- *BREAK_TIME* **->** The length of expert's break
- *BREAK_TIME_RATE* **->** The mean of Poisson distributed expert's break time. 8 breaks for 8 hour shift, so 1 break for 1 hour. 1/60 break for 1 minutes

In [2]:
RANDOM_SEED = 9780
INTERARRIVAL_MEAN = 15.6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

SERVICE_TIME_OPERATOR_MEAN = 4.2
SERVICE_TIME_OPERATOR_DEVIATION = 1.5

SERVICE_TIME_EXPERT_MEAN = 9.3
SERVICE_TIME_EXPERT_DEVIATION = 3.1
SERVICE_TIME_EXPERT_RATE = 1.0 / SERVICE_TIME_EXPERT_MEAN

BREAK_TIME = 5
BREAK_TIME_RATE = 1/60
CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

**Define the necessary set of arrays for statistics**

In [3]:
# Duration of the conversation between the customer and the operator (Operator service time)
service_times_operator = []

# Duration of the conversation between the customer and the expert (Expert service time)
service_times_expert = []

# Time spent by a customer while it waits for the operator (Operator Queue waiting time)
queue_w_times_operator = []

# Time spent by a customer while it waits for the expert (Expert Queue waiting time)
queue_w_times_expert = []

# Simulation end time
end_time = 0;

**Explanation of the process**

* The class definition for the customers arriving at the modeled system. When they are created, they immediatelly initiate a call (i.e. activate the call process). 

* Once a call is initiated, this is registered as a request to the operator resource. The customer is put on hold until the resource activates it back. 

* When the operator resource is available, the customer is activated and it then initiates the ask_question process. The duration of a question-answer session is determined randomly according to a uniform distribution.

* When the customer done with operator, he/she is registered as a request to the expert resource. The customer is put on hold until the resource activates it back. 

* When the expert resource is available, the customer is activated and it then initiates the get_help process. The duration of a help session is determined randomly according to a exponentially distribution.

In [4]:
class Customer(object):
    def __init__(self, name, env, operator, expert):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))

        with operator.request() as req:
            yield req
            print('%s is assigned to a operator at %g' % (self.name, self.env.now))
            queue_w_times_operator.append(self.env.now - self.arrival_t)
            yield self.env.process(self.ask_question())
            print('%s is done operator at %g' % (self.name, self.env.now))
            self.done_t = self.env.now
    
        with expert.request() as req:
            yield req
            print('%s is assigned to a expert at %g' % (self.name, self.env.now))
            queue_w_times_expert.append(self.env.now - self.done_t)
            yield self.env.process(self.get_help())
            print('%s is done expert at %g' % (self.name, self.env.now))
            global end_time 
            end_time = self.env.now

    def ask_question(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR_MEAN, SERVICE_TIME_OPERATOR_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR_MEAN, SERVICE_TIME_OPERATOR_DEVIATION)

        yield self.env.timeout(duration)
        service_times_operator.append(duration)
        
    def get_help(self):
        duration = random.expovariate(SERVICE_TIME_EXPERT_RATE)
        while duration < 0:
            duration = random.expovariate(SERVICE_TIME_EXPERT_RATE)
            
        yield self.env.timeout(duration)
        service_times_expert.append(duration)


In [5]:
class CustomerNorm(Customer):
    """ Customer that has normally distributed expert service"""
    def get_help(self):
        duration = random.gauss(SERVICE_TIME_EXPERT_MEAN, SERVICE_TIME_EXPERT_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_EXPERT_MEAN, SERVICE_TIME_EXPERT_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_expert.append(duration)



In [6]:
def customer_generator(env, operator, expert):
    """Generate new customer that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator, expert)  

In [7]:
def customer_norm_generator(env, operator, expert):
    """Generate new customer with normally distributed expert service that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = CustomerNorm('Customer %s' %(i+1), env, operator, expert)  

In [8]:
def break_generator(env, operator, expert):
    """Generate new break for expert."""
    while True:
        if len(queue_w_times_expert) == CUSTOMER_COUNT:
            break
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        with expert.request() as req:
            yield req
            print('Expert gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Expert break done at %g' % (env.now))



## Expert service time Exponentially distributed with a mean of 9.3 - 1000 CUSTOMER

In [9]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator, expert))
env.process(break_generator(env, operator, expert))
env.run()

Customer 1 initiated a call at 8.61921
Customer 1 is assigned to a operator at 8.61921
Customer 1 is done operator at 11.3398
Customer 1 is assigned to a expert at 11.3398
Customer 1 is done expert at 15.1787
Expert gives break at 17.5986
Expert break done at 22.5986
Expert gives break at 49.7269
Expert break done at 54.7269
Customer 2 initiated a call at 75.3283
Customer 2 is assigned to a operator at 75.3283
Customer 3 initiated a call at 76.252
Customer 2 is done operator at 80.9166
Customer 2 is assigned to a expert at 80.9166
Customer 3 is assigned to a operator at 80.9166
Customer 4 initiated a call at 82.6797
Customer 3 is done operator at 85.7185
Customer 4 is assigned to a operator at 85.7185
Customer 2 is done expert at 88.2612
Customer 3 is assigned to a expert at 88.2612
Customer 4 is done operator at 89.8649
Customer 3 is done expert at 92.1997
Customer 4 is assigned to a expert at 92.1997
Customer 4 is done expert at 93.5399
Customer 5 initiated a call at 105.961
Customer

Customer 197 is assigned to a expert at 3367.02
Customer 197 is done expert at 3372.69
Customer 198 initiated a call at 3411.3
Customer 198 is assigned to a operator at 3411.3
Customer 198 is done operator at 3415.58
Customer 198 is assigned to a expert at 3415.58
Customer 199 initiated a call at 3420.6
Customer 199 is assigned to a operator at 3420.6
Customer 199 is done operator at 3425.66
Customer 200 initiated a call at 3427.27
Customer 200 is assigned to a operator at 3427.27
Customer 200 is done operator at 3429.05
Customer 198 is done expert at 3430.22
Customer 199 is assigned to a expert at 3430.22
Customer 199 is done expert at 3430.28
Customer 200 is assigned to a expert at 3430.28
Customer 201 initiated a call at 3432.93
Customer 201 is assigned to a operator at 3432.93
Customer 201 is done operator at 3437.2
Customer 200 is done expert at 3440.25
Customer 201 is assigned to a expert at 3440.25
Customer 201 is done expert at 3444.71
Customer 202 initiated a call at 3457.49
C

Customer 509 initiated a call at 8085.63
Customer 510 initiated a call at 8085.97
Customer 502 is done expert at 8086.1
Customer 503 is assigned to a expert at 8086.1
Customer 503 is done expert at 8086.67
Customer 504 is assigned to a expert at 8086.67
Customer 506 is done operator at 8086.78
Customer 507 is assigned to a operator at 8086.78
Customer 507 is done operator at 8092.92
Customer 508 is assigned to a operator at 8092.92
Customer 508 is done operator at 8096.1
Customer 509 is assigned to a operator at 8096.1
Customer 509 is done operator at 8099.55
Customer 510 is assigned to a operator at 8099.55
Customer 511 initiated a call at 8101.8
Customer 512 initiated a call at 8101.96
Customer 510 is done operator at 8103.85
Customer 511 is assigned to a operator at 8103.85
Customer 511 is done operator at 8109.09
Customer 512 is assigned to a operator at 8109.09
Customer 512 is done operator at 8113.6
Customer 504 is done expert at 8116.01
Customer 505 is assigned to a expert at 81

Customer 864 is done operator at 13915.1
Expert break done at 13915.4
Customer 864 is assigned to a expert at 13915.4
Customer 865 initiated a call at 13917.6
Customer 865 is assigned to a operator at 13917.6
Customer 865 is done operator at 13921
Customer 866 initiated a call at 13929.3
Customer 866 is assigned to a operator at 13929.3
Customer 867 initiated a call at 13931.3
Customer 866 is done operator at 13932.2
Customer 867 is assigned to a operator at 13932.2
Customer 867 is done operator at 13935
Customer 864 is done expert at 13942.8
Customer 865 is assigned to a expert at 13942.8
Customer 865 is done expert at 13943.2
Customer 866 is assigned to a expert at 13943.2
Customer 866 is done expert at 13943.2
Customer 867 is assigned to a expert at 13943.2
Customer 867 is done expert at 13962.7
Customer 868 initiated a call at 13987.1
Customer 868 is assigned to a operator at 13987.1
Customer 868 is done operator at 13990.2
Customer 868 is assigned to a expert at 13990.2
Customer 8

In [10]:
# Calculating statistics
CUSTOMER_COUNT = 1000
print("Expert service time Exponentially distributed with a mean of 9.3 - 1000 CUSTOMER")
total_operator_service_time = sum(service_times_operator)
operator_util = total_operator_service_time / end_time
total_expert_service_time = sum(service_times_expert)
expert_util = total_expert_service_time / end_time
print("First Operator Utilization: %f" % (operator_util))
print("Expert Utilization: %f" % (expert_util))

ave_total_wait_time = sum([x + y for x, y in zip(queue_w_times_operator, queue_w_times_expert)]) / CUSTOMER_COUNT
print("Average Total Waiting Time: %f" % (ave_total_wait_time))

max_total_w_system_time_ratio = max([(w1 + w2) / end_time for w1, w2 in zip(queue_w_times_operator, queue_w_times_expert)])
print("Maximum Total Waiting Time to Total System Time Ratio: %f" % (max_total_w_system_time_ratio))

ave_people_w_expert = sum(queue_w_times_expert) / end_time
print("Average Number of people waiting to be served by the expert operator: %f" % (ave_people_w_expert))


Expert service time Exponentially distributed with a mean of 9.3 - 1000 CUSTOMER
First Operator Utilization: 0.263762
Expert Utilization: 0.567710
Average Total Waiting Time: 12.439631
Maximum Total Waiting Time to Total System Time Ratio: 0.005287
Average Number of people waiting to be served by the expert operator: 0.709159


## Expert service time Exponentially distributed with a mean of 9.3 - 5000 CUSTOMER

In [11]:
# Reset statistics
service_times_operator = []
service_times_expert = []
queue_w_times_operator = []
queue_w_times_expert = []
end_time = 0;

# Change customer count
CUSTOMER_COUNT = 5000

In [12]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator, expert))
env.process(break_generator(env, operator, expert))
env.run()

Customer 1 initiated a call at 28.0454
Customer 1 is assigned to a operator at 28.0454
Customer 1 is done operator at 31.7669
Customer 1 is assigned to a expert at 31.7669
Customer 1 is done expert at 52.1527
Expert gives break at 52.1527
Customer 2 initiated a call at 53.4391
Customer 2 is assigned to a operator at 53.4391
Customer 2 is done operator at 56.2892
Expert break done at 57.1527
Customer 2 is assigned to a expert at 57.1527
Customer 2 is done expert at 62.2976
Customer 3 initiated a call at 66.2138
Customer 3 is assigned to a operator at 66.2138
Customer 3 is done operator at 69.1361
Customer 3 is assigned to a expert at 69.1361
Customer 3 is done expert at 81.4069
Customer 4 initiated a call at 117.502
Customer 4 is assigned to a operator at 117.502
Customer 4 is done operator at 122.922
Customer 4 is assigned to a expert at 122.922
Customer 4 is done expert at 123.719
Customer 5 initiated a call at 136.389
Customer 5 is assigned to a operator at 136.389
Customer 5 is done

Customer 120 is done operator at 2263.49
Customer 119 is done expert at 2263.73
Customer 120 is assigned to a expert at 2263.73
Customer 120 is done expert at 2268.85
Customer 121 initiated a call at 2271.21
Customer 121 is assigned to a operator at 2271.21
Customer 121 is done operator at 2275.78
Customer 121 is assigned to a expert at 2275.78
Customer 122 initiated a call at 2288.84
Customer 122 is assigned to a operator at 2288.84
Customer 122 is done operator at 2292.07
Customer 121 is done expert at 2302.76
Expert gives break at 2302.76
Expert break done at 2307.76
Customer 122 is assigned to a expert at 2307.76
Customer 122 is done expert at 2307.93
Expert gives break at 2309.89
Expert break done at 2314.89
Customer 123 initiated a call at 2319.03
Customer 123 is assigned to a operator at 2319.03
Customer 123 is done operator at 2323.66
Customer 123 is assigned to a expert at 2323.66
Customer 123 is done expert at 2330.63
Customer 124 initiated a call at 2334.33
Customer 124 is a

Customer 170 initiated a call at 3096.44
Customer 170 is assigned to a operator at 3096.44
Customer 170 is done operator at 3098.86
Customer 171 initiated a call at 3105.31
Customer 171 is assigned to a operator at 3105.31
Customer 169 is done expert at 3105.31
Customer 170 is assigned to a expert at 3105.31
Customer 172 initiated a call at 3105.7
Customer 173 initiated a call at 3107.19
Customer 170 is done expert at 3108.05
Customer 171 is done operator at 3111.15
Customer 171 is assigned to a expert at 3111.15
Customer 172 is assigned to a operator at 3111.15
Customer 172 is done operator at 3112.73
Customer 173 is assigned to a operator at 3112.73
Customer 171 is done expert at 3116.05
Customer 172 is assigned to a expert at 3116.05
Customer 173 is done operator at 3116.41
Customer 172 is done expert at 3128.94
Customer 173 is assigned to a expert at 3128.94
Customer 173 is done expert at 3133.86
Expert gives break at 3133.86
Expert break done at 3138.86
Customer 174 initiated a ca

Customer 408 is done expert at 6555.02
Customer 409 is assigned to a expert at 6555.02
Customer 409 is done expert at 6557.33
Customer 410 is assigned to a expert at 6557.33
Customer 410 is done expert at 6557.6
Expert gives break at 6557.6
Expert break done at 6562.6
Customer 411 initiated a call at 6571.12
Customer 411 is assigned to a operator at 6571.12
Customer 411 is done operator at 6577.54
Customer 411 is assigned to a expert at 6577.54
Customer 411 is done expert at 6582.84
Expert gives break at 6591.92
Expert break done at 6596.92
Customer 412 initiated a call at 6622.27
Customer 412 is assigned to a operator at 6622.27
Customer 412 is done operator at 6625.58
Customer 412 is assigned to a expert at 6625.58
Customer 412 is done expert at 6632.05
Customer 413 initiated a call at 6638.18
Customer 413 is assigned to a operator at 6638.18
Customer 413 is done operator at 6645.91
Customer 413 is assigned to a expert at 6645.91
Customer 413 is done expert at 6646.14
Customer 414 in

Customer 474 initiated a call at 7615.4
Customer 474 is assigned to a operator at 7615.4
Customer 475 initiated a call at 7617.28
Customer 474 is done operator at 7619.11
Customer 475 is assigned to a operator at 7619.11
Customer 475 is done operator at 7625.02
Customer 471 is done expert at 7636.72
Customer 472 is assigned to a expert at 7636.72
Customer 472 is done expert at 7638.07
Customer 473 is assigned to a expert at 7638.07
Customer 473 is done expert at 7638.83
Customer 474 is assigned to a expert at 7638.83
Customer 474 is done expert at 7648.52
Expert gives break at 7648.52
Expert break done at 7653.52
Customer 475 is assigned to a expert at 7653.52
Customer 476 initiated a call at 7655.75
Customer 476 is assigned to a operator at 7655.75
Customer 477 initiated a call at 7656.11
Customer 476 is done operator at 7660.76
Customer 477 is assigned to a operator at 7660.76
Customer 477 is done operator at 7663.75
Customer 475 is done expert at 7665.6
Customer 476 is assigned to a

...

Customer 3797 is assigned to a expert at 59267
Customer 3797 is done expert at 59267.6
Customer 3798 is assigned to a expert at 59267.6
Customer 3798 is done expert at 59272.8
Expert gives break at 59279
Customer 3799 initiated a call at 59279.4
Customer 3799 is assigned to a operator at 59279.4
Expert break done at 59284
Customer 3799 is done operator at 59284.8
Customer 3799 is assigned to a expert at 59284.8
Customer 3800 initiated a call at 59285
Customer 3800 is assigned to a operator at 59285
Customer 3799 is done expert at 59285.3
Customer 3800 is done operator at 59286.7
Customer 3800 is assigned to a expert at 59286.7
Customer 3800 is done expert at 59287.7
Customer 3801 initiated a call at 59291.3
Customer 3801 is assigned to a operator at 59291.3
Customer 3801 is done operator at 59298.2
Customer 3801 is assigned to a expert at 59298.2
Customer 3801 is done expert at 59299
Customer 3802 initiated a call at 59336.6
Customer 3802 is assigned to a operator at 59336.6
Customer 3

Customer 4094 is done operator at 63904.7
Customer 4094 is assigned to a expert at 63904.7
Customer 4094 is done expert at 63909.8
Customer 4095 initiated a call at 63924.6
Customer 4095 is assigned to a operator at 63924.6
Customer 4096 initiated a call at 63925.3
Customer 4095 is done operator at 63930.4
Customer 4095 is assigned to a expert at 63930.4
Customer 4096 is assigned to a operator at 63930.4
Customer 4096 is done operator at 63933.6
Customer 4097 initiated a call at 63935
Customer 4097 is assigned to a operator at 63935
Customer 4095 is done expert at 63935.7
Customer 4096 is assigned to a expert at 63935.7
Customer 4097 is done operator at 63938.8
Customer 4096 is done expert at 63956.5
Customer 4097 is assigned to a expert at 63956.5
Customer 4098 initiated a call at 63961
Customer 4098 is assigned to a operator at 63961
Customer 4097 is done expert at 63961.8
Customer 4098 is done operator at 63965.1
Customer 4098 is assigned to a expert at 63965.1
Customer 4099 initiat

Customer 4443 is done operator at 69501.7
Customer 4443 is assigned to a expert at 69501.7
Customer 4444 initiated a call at 69502.3
Customer 4444 is assigned to a operator at 69502.3
Customer 4444 is done operator at 69507.9
Customer 4445 initiated a call at 69520.2
Customer 4445 is assigned to a operator at 69520.2
Customer 4445 is done operator at 69524.5
Customer 4443 is done expert at 69542.5
Customer 4444 is assigned to a expert at 69542.5
Customer 4444 is done expert at 69543.4
Customer 4445 is assigned to a expert at 69543.4
Customer 4445 is done expert at 69545
Expert gives break at 69545
Expert break done at 69550
Customer 4446 initiated a call at 69564.9
Customer 4446 is assigned to a operator at 69564.9
Customer 4446 is done operator at 69568.2
Customer 4446 is assigned to a expert at 69568.2
Customer 4446 is done expert at 69604
Customer 4447 initiated a call at 69618.5
Customer 4447 is assigned to a operator at 69618.5
Customer 4448 initiated a call at 69622.5
Customer 44

Customer 4766 is assigned to a operator at 74489.5
Expert gives break at 74490.6
Customer 4767 initiated a call at 74490.8
Expert break done at 74495.6
Customer 4766 is done operator at 74496.2
Customer 4766 is assigned to a expert at 74496.2
Customer 4767 is assigned to a operator at 74496.2
Customer 4766 is done expert at 74500.2
Customer 4767 is done operator at 74500.8
Customer 4767 is assigned to a expert at 74500.8
Customer 4767 is done expert at 74504.4
Customer 4768 initiated a call at 74506
Customer 4768 is assigned to a operator at 74506
Customer 4768 is done operator at 74510
Customer 4768 is assigned to a expert at 74510
Customer 4768 is done expert at 74540.7
Expert gives break at 74540.7
Expert break done at 74545.7
Expert gives break at 74571.7
Customer 4769 initiated a call at 74576.3
Customer 4769 is assigned to a operator at 74576.3
Expert break done at 74576.7
Customer 4769 is done operator at 74579.7
Customer 4769 is assigned to a expert at 74579.7
Customer 4769 is 

Customer 4804 is done operator at 75061.4
Customer 4804 is assigned to a expert at 75061.4
Customer 4804 is done expert at 75068.5
Customer 4805 initiated a call at 75081.1
Customer 4805 is assigned to a operator at 75081.1
Customer 4806 initiated a call at 75084.4
Customer 4805 is done operator at 75087.6
Customer 4805 is assigned to a expert at 75087.6
Customer 4806 is assigned to a operator at 75087.6
Customer 4807 initiated a call at 75090.6
Customer 4806 is done operator at 75091.5
Customer 4807 is assigned to a operator at 75091.5
Customer 4805 is done expert at 75091.8
Customer 4806 is assigned to a expert at 75091.8
Customer 4808 initiated a call at 75093.2
Customer 4809 initiated a call at 75093.3
Customer 4807 is done operator at 75095.6
Customer 4808 is assigned to a operator at 75095.6
Customer 4806 is done expert at 75096.4
Customer 4807 is assigned to a expert at 75096.4
Customer 4807 is done expert at 75096.7
Customer 4808 is done operator at 75098.7
Customer 4808 is ass

In [13]:
# Calculating statistics
CUSTOMER_COUNT = 5000
print("Expert service time Exponentially distributed with a mean of 9.3 - 5000 CUSTOMER")
total_operator_service_time = sum(service_times_operator)
operator_util = total_operator_service_time / end_time
total_expert_service_time = sum(service_times_expert)
expert_util = total_expert_service_time / end_time
print("First Operator Utilization: %f" % (operator_util))
print("Expert Utilization: %f" % (expert_util))

ave_total_wait_time = sum([x + y for x, y in zip(queue_w_times_operator, queue_w_times_expert)]) / CUSTOMER_COUNT
print("Average Total Waiting Time: %f" % (ave_total_wait_time))

max_total_w_system_time_ratio = max([(w1 + w2) / end_time for w1, w2 in zip(queue_w_times_operator, queue_w_times_expert)])
print("Maximum Total Waiting Time to Total System Time Ratio: %f" % (max_total_w_system_time_ratio))

ave_people_w_expert = sum(queue_w_times_expert) / end_time
print("Average Number of people waiting to be served by the expert operator: %f" % (ave_people_w_expert))

Expert service time Exponentially distributed with a mean of 9.3 - 5000 CUSTOMER
First Operator Utilization: 0.272887
Expert Utilization: 0.598772
Average Total Waiting Time: 16.079783
Maximum Total Waiting Time to Total System Time Ratio: 0.001861
Average Number of people waiting to be served by the expert operator: 0.976480


## Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 1000 Customer

In [14]:
# Reset statistics
service_times_operator = []
service_times_expert = []
queue_w_times_operator = []
queue_w_times_expert = []
end_time = 0;

# Change customer count
CUSTOMER_COUNT = 1000

In [15]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert = simpy.Resource(env, capacity = 1)
env.process(customer_norm_generator(env, operator, expert))
env.process(break_generator(env, operator, expert))
env.run()

Customer 1 initiated a call at 25.37
Customer 1 is assigned to a operator at 25.37
Customer 1 is done operator at 32.9984
Customer 1 is assigned to a expert at 32.9984
Customer 2 initiated a call at 35.1219
Customer 2 is assigned to a operator at 35.1219
Customer 2 is done operator at 39.6838
Customer 1 is done expert at 41.4578
Customer 2 is assigned to a expert at 41.4578
Customer 2 is done expert at 48.5008
Expert gives break at 63.5818
Expert break done at 68.5818
Customer 3 initiated a call at 81.018
Customer 3 is assigned to a operator at 81.018
Customer 4 initiated a call at 82.2862
Customer 3 is done operator at 85.4621
Customer 3 is assigned to a expert at 85.4621
Customer 4 is assigned to a operator at 85.4621
Customer 4 is done operator at 90.0385
Customer 3 is done expert at 94.541
Customer 4 is assigned to a expert at 94.541
Customer 4 is done expert at 108.502
Expert gives break at 108.502
Expert break done at 113.502
Customer 5 initiated a call at 118.082
Customer 5 is a

Customer 96 is assigned to a operator at 1376.86
Customer 94 is done expert at 1377.91
Customer 95 is assigned to a expert at 1377.91
Customer 96 is done operator at 1380.63
Customer 95 is done expert at 1386.82
Customer 96 is assigned to a expert at 1386.82
Customer 97 initiated a call at 1393.1
Customer 97 is assigned to a operator at 1393.1
Customer 96 is done expert at 1396.31
Customer 97 is done operator at 1398.15
Customer 97 is assigned to a expert at 1398.15
Customer 97 is done expert at 1407.98
Customer 98 initiated a call at 1408.58
Customer 98 is assigned to a operator at 1408.58
Customer 98 is done operator at 1414.92
Customer 98 is assigned to a expert at 1414.92
Customer 99 initiated a call at 1425.83
Customer 99 is assigned to a operator at 1425.83
Customer 100 initiated a call at 1426.42
Customer 98 is done expert at 1427.29
Customer 99 is done operator at 1430.91
Customer 99 is assigned to a expert at 1430.91
Customer 100 is assigned to a operator at 1430.91
Customer 1

Customer 155 is done expert at 2359.64
Customer 156 is assigned to a expert at 2359.64
Customer 156 is done expert at 2370.34
Customer 157 initiated a call at 2382.57
Customer 157 is assigned to a operator at 2382.57
Customer 157 is done operator at 2387.33
Customer 157 is assigned to a expert at 2387.33
Customer 157 is done expert at 2395.51
Customer 158 initiated a call at 2404.9
Customer 158 is assigned to a operator at 2404.9
Customer 159 initiated a call at 2405.5
Customer 160 initiated a call at 2407.26
Customer 158 is done operator at 2410.55
Customer 158 is assigned to a expert at 2410.55
Customer 159 is assigned to a operator at 2410.55
Customer 159 is done operator at 2415.66
Customer 160 is assigned to a operator at 2415.66
Customer 160 is done operator at 2419.39
Customer 158 is done expert at 2422.21
Customer 159 is assigned to a expert at 2422.21
Customer 159 is done expert at 2428.3
Customer 160 is assigned to a expert at 2428.3
Customer 160 is done expert at 2435.98
Cus

Customer 371 is assigned to a expert at 5923.81
Customer 371 is done expert at 5931.65
Customer 372 initiated a call at 5934.05
Customer 372 is assigned to a operator at 5934.05
Customer 372 is done operator at 5939.76
Customer 372 is assigned to a expert at 5939.76
Customer 372 is done expert at 5949.48
Expert gives break at 5949.48
Expert break done at 5954.48
Expert gives break at 5956.69
Expert break done at 5961.69
Customer 373 initiated a call at 5968.81
Customer 373 is assigned to a operator at 5968.81
Customer 373 is done operator at 5970.82
Customer 373 is assigned to a expert at 5970.82
Customer 374 initiated a call at 5976.07
Customer 374 is assigned to a operator at 5976.07
Customer 373 is done expert at 5976.08
Expert gives break at 5977.44
Customer 374 is done operator at 5979.29
Expert break done at 5982.44
Customer 374 is assigned to a expert at 5982.44
Customer 374 is done expert at 5991.49
Customer 375 initiated a call at 6016.29
Customer 375 is assigned to a operator

Customer 472 is done operator at 7359.22
Expert break done at 7361.72
Customer 471 is assigned to a expert at 7361.72
Customer 473 initiated a call at 7366.79
Customer 473 is assigned to a operator at 7366.79
Customer 473 is done operator at 7371
Customer 471 is done expert at 7372.71
Customer 472 is assigned to a expert at 7372.71
Customer 472 is done expert at 7378.69
Customer 473 is assigned to a expert at 7378.69
Customer 473 is done expert at 7389.47
Customer 474 initiated a call at 7425.19
Customer 474 is assigned to a operator at 7425.19
Customer 474 is done operator at 7430.02
Customer 474 is assigned to a expert at 7430.02
Customer 474 is done expert at 7436.92
Customer 475 initiated a call at 7448.47
Customer 475 is assigned to a operator at 7448.47
Customer 475 is done operator at 7453.01
Customer 475 is assigned to a expert at 7453.01
Customer 476 initiated a call at 7461.04
Customer 476 is assigned to a operator at 7461.04
Customer 477 initiated a call at 7461.3
Customer 4

Customer 693 is assigned to a operator at 10711.2
Customer 693 is done operator at 10714.6
Customer 692 is done expert at 10719.4
Customer 693 is assigned to a expert at 10719.4
Customer 693 is done expert at 10728.7
Expert gives break at 10728.7
Customer 694 initiated a call at 10729.3
Customer 694 is assigned to a operator at 10729.3
Customer 695 initiated a call at 10731.7
Customer 694 is done operator at 10733
Customer 695 is assigned to a operator at 10733
Expert break done at 10733.7
Customer 694 is assigned to a expert at 10733.7
Customer 695 is done operator at 10736.2
Customer 694 is done expert at 10749.5
Customer 695 is assigned to a expert at 10749.5
Customer 696 initiated a call at 10758.4
Customer 696 is assigned to a operator at 10758.4
Customer 697 initiated a call at 10760.3
Customer 696 is done operator at 10760.8
Customer 697 is assigned to a operator at 10760.8
Customer 695 is done expert at 10762.5
Expert gives break at 10762.5
Expert break done at 10767.5
Customer

Customer 744 is done expert at 11564.8
Customer 745 initiated a call at 11567.8
Customer 745 is assigned to a operator at 11567.8
Customer 745 is done operator at 11572
Customer 745 is assigned to a expert at 11572
Customer 746 initiated a call at 11574.6
Customer 746 is assigned to a operator at 11574.6
Customer 746 is done operator at 11578.3
Customer 745 is done expert at 11581.9
Customer 746 is assigned to a expert at 11581.9
Customer 746 is done expert at 11590.4
Customer 747 initiated a call at 11592.2
Customer 747 is assigned to a operator at 11592.2
Expert gives break at 11595.8
Customer 747 is done operator at 11598.3
Expert break done at 11600.8
Customer 747 is assigned to a expert at 11600.8
Customer 747 is done expert at 11610
Expert gives break at 11610
Expert break done at 11615
Customer 748 initiated a call at 11619
Customer 748 is assigned to a operator at 11619
Customer 749 initiated a call at 11620.1
Customer 748 is done operator at 11622.4
Customer 748 is assigned to

Customer 818 is done operator at 12685.9
Customer 817 is done expert at 12687.2
Expert gives break at 12687.2
Expert break done at 12692.2
Customer 818 is assigned to a expert at 12692.2
Customer 818 is done expert at 12700.7
Expert gives break at 12707.7
Expert break done at 12712.7
Customer 819 initiated a call at 12714.1
Customer 819 is assigned to a operator at 12714.1
Customer 819 is done operator at 12718
Customer 819 is assigned to a expert at 12718
Customer 820 initiated a call at 12721.1
Customer 820 is assigned to a operator at 12721.1
Customer 820 is done operator at 12725.7
Customer 821 initiated a call at 12725.7
Customer 821 is assigned to a operator at 12725.7
Customer 822 initiated a call at 12725.9
Customer 823 initiated a call at 12726.1
Customer 819 is done expert at 12728
Customer 820 is assigned to a expert at 12728
Customer 821 is done operator at 12729.8
Customer 822 is assigned to a operator at 12729.8
Customer 822 is done operator at 12734.8
Customer 823 is ass

In [16]:
# Calculating statistics

CUSTOMER_COUNT = 1000
print("Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 1000 CUSTOMER")
total_operator_service_time = sum(service_times_operator)
operator_util = total_operator_service_time / end_time
total_expert_service_time = sum(service_times_expert)
expert_util = total_expert_service_time / end_time
print("First Operator Utilization: %f" % (operator_util))
print("Expert Utilization: %f" % (expert_util))

ave_total_wait_time = sum([x + y for x, y in zip(queue_w_times_operator, queue_w_times_expert)]) / CUSTOMER_COUNT
print("Average Total Waiting Time: %f" % (ave_total_wait_time))

max_total_w_system_time_ratio = max([(w1 + w2) / end_time for w1, w2 in zip(queue_w_times_operator, queue_w_times_expert)])
print("Maximum Total Waiting Time to Total System Time Ratio: %f" % (max_total_w_system_time_ratio))

ave_people_w_expert = sum(queue_w_times_expert) / end_time
print("Average Number of people waiting to be served by the expert operator: %f" % (ave_people_w_expert))

Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 1000 CUSTOMER
First Operator Utilization: 0.278743
Expert Utilization: 0.612020
Average Total Waiting Time: 8.128721
Maximum Total Waiting Time to Total System Time Ratio: 0.003692
Average Number of people waiting to be served by the expert operator: 0.474471


## Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 5000 Customer

In [17]:
# Reset statistics
service_times_operator = []
service_times_expert = []
queue_w_times_operator = []
queue_w_times_expert = []
end_time = 0;

# Change customer count
CUSTOMER_COUNT = 5000

In [18]:
env = simpy.Environment()
operator = simpy.Resource(env, capacity = 1)
expert = simpy.Resource(env, capacity = 1)
env.process(customer_norm_generator(env, operator, expert))
env.process(break_generator(env, operator, expert))
env.run()

Expert gives break at 15.6938
Customer 1 initiated a call at 17.1236
Customer 1 is assigned to a operator at 17.1236
Expert break done at 20.6938
Customer 1 is done operator at 24.0585
Customer 1 is assigned to a expert at 24.0585
Customer 2 initiated a call at 30.3964
Customer 2 is assigned to a operator at 30.3964
Customer 1 is done expert at 32.8045
Customer 3 initiated a call at 34.1342
Customer 2 is done operator at 35.9563
Customer 2 is assigned to a expert at 35.9563
Customer 3 is assigned to a operator at 35.9563
Customer 4 initiated a call at 39.9428
Customer 3 is done operator at 40.7676
Customer 4 is assigned to a operator at 40.7676
Customer 4 is done operator at 45.761
Customer 2 is done expert at 52.4748
Customer 3 is assigned to a expert at 52.4748
Customer 5 initiated a call at 54.7215
Customer 5 is assigned to a operator at 54.7215
Customer 3 is done expert at 57.0569
Customer 4 is assigned to a expert at 57.0569
Customer 5 is done operator at 60.8617
Customer 4 is don

Customer 67 initiated a call at 1175.77
Customer 67 is assigned to a operator at 1175.77
Customer 67 is done operator at 1179.93
Customer 66 is done expert at 1179.97
Customer 67 is assigned to a expert at 1179.97
Customer 67 is done expert at 1186.32
Expert gives break at 1193.52
Expert break done at 1198.52
Customer 68 initiated a call at 1210.18
Customer 68 is assigned to a operator at 1210.18
Customer 69 initiated a call at 1212.55
Customer 68 is done operator at 1214.1
Customer 68 is assigned to a expert at 1214.1
Customer 69 is assigned to a operator at 1214.1
Customer 69 is done operator at 1218.37
Customer 68 is done expert at 1227.14
Customer 69 is assigned to a expert at 1227.14
Customer 69 is done expert at 1234.71
Customer 70 initiated a call at 1247.03
Customer 70 is assigned to a operator at 1247.03
Customer 70 is done operator at 1249.93
Customer 70 is assigned to a expert at 1249.93
Customer 70 is done expert at 1257.72
Customer 71 initiated a call at 1275.03
Customer 7

Customer 109 is done operator at 1844.39
Customer 109 is assigned to a expert at 1844.39
Customer 110 is assigned to a operator at 1844.39
Customer 110 is done operator at 1850.85
Customer 111 initiated a call at 1853.59
Customer 111 is assigned to a operator at 1853.59
Customer 111 is done operator at 1856.26
Customer 109 is done expert at 1856.73
Customer 110 is assigned to a expert at 1856.73
Customer 112 initiated a call at 1856.81
Customer 112 is assigned to a operator at 1856.81
Customer 112 is done operator at 1863.34
Customer 110 is done expert at 1865.63
Customer 111 is assigned to a expert at 1865.63
Customer 113 initiated a call at 1866.65
Customer 113 is assigned to a operator at 1866.65
Customer 113 is done operator at 1869.78
Customer 114 initiated a call at 1873.33
Customer 114 is assigned to a operator at 1873.33
Customer 111 is done expert at 1873.42
Customer 112 is assigned to a expert at 1873.42
Customer 114 is done operator at 1878.87
Customer 112 is done expert at 

Customer 157 is done expert at 2624.53
Customer 158 is assigned to a expert at 2624.53
Customer 161 initiated a call at 2625.41
Customer 161 is assigned to a operator at 2625.41
Customer 161 is done operator at 2627.76
Customer 158 is done expert at 2636.37
Expert gives break at 2636.37
Expert break done at 2641.37
Customer 159 is assigned to a expert at 2641.37
Customer 162 initiated a call at 2645.66
Customer 162 is assigned to a operator at 2645.66
Customer 162 is done operator at 2649.64
Customer 159 is done expert at 2652.11
Customer 160 is assigned to a expert at 2652.11
Customer 160 is done expert at 2660.54
Customer 161 is assigned to a expert at 2660.54
Customer 163 initiated a call at 2663.47
Customer 163 is assigned to a operator at 2663.47
Customer 163 is done operator at 2666.76
Customer 161 is done expert at 2668.8
Customer 162 is assigned to a expert at 2668.8
Customer 162 is done expert at 2681.34
Expert gives break at 2681.34
Customer 164 initiated a call at 2682.72
Cu

Expert gives break at 3773.63
Customer 228 initiated a call at 3774.72
Customer 227 is done operator at 3776.61
Customer 228 is assigned to a operator at 3776.61
Expert break done at 3778.63
Customer 226 is assigned to a expert at 3778.63
Customer 228 is done operator at 3782.44
Customer 226 is done expert at 3788.44
Customer 227 is assigned to a expert at 3788.44
Customer 227 is done expert at 3794.37
Customer 228 is assigned to a expert at 3794.37
Customer 229 initiated a call at 3800.93
Customer 229 is assigned to a operator at 3800.93
Customer 230 initiated a call at 3801.97
Customer 231 initiated a call at 3803.51
Customer 229 is done operator at 3805.26
Customer 230 is assigned to a operator at 3805.26
Customer 228 is done expert at 3808.41
Customer 229 is assigned to a expert at 3808.41
Customer 230 is done operator at 3808.76
Customer 231 is assigned to a operator at 3808.76
Customer 232 initiated a call at 3810.43
Customer 231 is done operator at 3812.88
Customer 232 is assign

...

Customer 4487 initiated a call at 70322.6
Customer 4487 is assigned to a operator at 70322.6
Customer 4487 is done operator at 70327.3
Customer 4487 is assigned to a expert at 70327.3
Customer 4487 is done expert at 70337.6
Customer 4488 initiated a call at 70347.9
Customer 4488 is assigned to a operator at 70347.9
Customer 4488 is done operator at 70352.8
Customer 4488 is assigned to a expert at 70352.8
Customer 4488 is done expert at 70363.6
Expert gives break at 70363.6
Customer 4489 initiated a call at 70364.9
Customer 4489 is assigned to a operator at 70364.9
Customer 4490 initiated a call at 70365.6
Customer 4491 initiated a call at 70367.9
Expert break done at 70368.6
Customer 4489 is done operator at 70370
Customer 4489 is assigned to a expert at 70370
Customer 4490 is assigned to a operator at 70370
Customer 4490 is done operator at 70372.3
Customer 4491 is assigned to a operator at 70372.3
Customer 4491 is done operator at 70373.9
Customer 4489 is done expert at 70380.1
Custo

Expert gives break at 72073.1
Expert break done at 72078.1
Customer 4624 is assigned to a expert at 72078.1
Customer 4624 is done expert at 72086.9
Expert gives break at 72086.9
Expert break done at 72091.9
Customer 4625 initiated a call at 72096.8
Customer 4625 is assigned to a operator at 72096.8
Customer 4625 is done operator at 72099.8
Customer 4625 is assigned to a expert at 72099.8
Customer 4625 is done expert at 72101.4
Customer 4626 initiated a call at 72116.6
Customer 4626 is assigned to a operator at 72116.6
Customer 4627 initiated a call at 72118.1
Customer 4626 is done operator at 72120.1
Customer 4626 is assigned to a expert at 72120.1
Customer 4627 is assigned to a operator at 72120.1
Customer 4627 is done operator at 72125
Customer 4628 initiated a call at 72126.9
Customer 4628 is assigned to a operator at 72126.9
Customer 4626 is done expert at 72129.9
Customer 4627 is assigned to a expert at 72129.9
Customer 4628 is done operator at 72131.1
Customer 4627 is done expert

Customer 4675 initiated a call at 72836.8
Customer 4675 is assigned to a operator at 72836.8
Customer 4676 initiated a call at 72838.2
Customer 4674 is done expert at 72842.1
Expert gives break at 72842.1
Customer 4675 is done operator at 72844.2
Customer 4676 is assigned to a operator at 72844.2
Customer 4676 is done operator at 72846.4
Expert break done at 72847.1
Customer 4675 is assigned to a expert at 72847.1
Customer 4675 is done expert at 72859.9
Customer 4676 is assigned to a expert at 72859.9
Customer 4676 is done expert at 72869.3
Expert gives break at 72869.3
Expert break done at 72874.3
Customer 4677 initiated a call at 72876.3
Customer 4677 is assigned to a operator at 72876.3
Customer 4677 is done operator at 72881.1
Customer 4677 is assigned to a expert at 72881.1
Customer 4678 initiated a call at 72881.3
Customer 4678 is assigned to a operator at 72881.3
Customer 4678 is done operator at 72883.9
Customer 4679 initiated a call at 72891.7
Customer 4679 is assigned to a op

Customer 4722 is assigned to a operator at 73454.3
Customer 4723 initiated a call at 73455.9
Customer 4722 is done operator at 73457.8
Customer 4723 is assigned to a operator at 73457.8
Expert break done at 73459.3
Customer 4721 is assigned to a expert at 73459.3
Customer 4723 is done operator at 73463.5
Customer 4721 is done expert at 73469
Customer 4722 is assigned to a expert at 73469
Customer 4724 initiated a call at 73472.3
Customer 4724 is assigned to a operator at 73472.3
Customer 4724 is done operator at 73474.3
Customer 4722 is done expert at 73482.5
Customer 4723 is assigned to a expert at 73482.5
Customer 4723 is done expert at 73494.1
Customer 4724 is assigned to a expert at 73494.1
Customer 4724 is done expert at 73509
Customer 4725 initiated a call at 73511.1
Customer 4725 is assigned to a operator at 73511.1
Customer 4725 is done operator at 73515.1
Customer 4725 is assigned to a expert at 73515.1
Customer 4726 initiated a call at 73524.1
Customer 4726 is assigned to a o

Customer 4833 is done operator at 74973.6
Customer 4831 is done expert at 74977
Customer 4832 is assigned to a expert at 74977
Customer 4832 is done expert at 74988.7
Customer 4833 is assigned to a expert at 74988.7
Customer 4833 is done expert at 74996.5
Customer 4834 initiated a call at 74997
Customer 4834 is assigned to a operator at 74997
Customer 4834 is done operator at 75000
Customer 4834 is assigned to a expert at 75000
Customer 4834 is done expert at 75009.7
Customer 4835 initiated a call at 75044.4
Customer 4835 is assigned to a operator at 75044.4
Customer 4836 initiated a call at 75048.4
Customer 4835 is done operator at 75049.2
Customer 4835 is assigned to a expert at 75049.2
Customer 4836 is assigned to a operator at 75049.2
Customer 4837 initiated a call at 75051.3
Customer 4836 is done operator at 75055.2
Customer 4837 is assigned to a operator at 75055.2
Customer 4835 is done expert at 75059.3
Customer 4836 is assigned to a expert at 75059.3
Customer 4837 is done opera

In [19]:
# Calculating statistics

CUSTOMER_COUNT = 5000
print("Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 5000 CUSTOMER")
total_operator_service_time = sum(service_times_operator)
operator_util = total_operator_service_time / end_time
total_expert_service_time = sum(service_times_expert)
expert_util = total_expert_service_time / end_time
print("First Operator Utilization: %f" % (operator_util))
print("Expert Utilization: %f" % (expert_util))

ave_total_wait_time = sum([x + y for x, y in zip(queue_w_times_operator, queue_w_times_expert)]) / CUSTOMER_COUNT
print("Average Total Waiting Time: %f" % (ave_total_wait_time))

max_total_w_system_time_ratio = max([(w1 + w2) / end_time for w1, w2 in zip(queue_w_times_operator, queue_w_times_expert)])
print("Maximum Total Waiting Time to Total System Time Ratio: %f" % (max_total_w_system_time_ratio))

ave_people_w_expert = sum(queue_w_times_expert) / end_time
print("Average Number of people waiting to be served by the expert operator: %f" % (ave_people_w_expert))

Expert service time Normally distributed with a mean of 9.3 and with a standard deviation of 3.1 - 5000 CUSTOMER
First Operator Utilization: 0.271832
Expert Utilization: 0.596906
Average Total Waiting Time: 8.652555
Maximum Total Waiting Time to Total System Time Ratio: 0.001170
Average Number of people waiting to be served by the expert operator: 0.499463
